# Perceptual Loss For Real Time Transfer

该网络与原始版本的Style Transfer的套路是一样的，唯一的不同就是原始版本的style transfer是直接拿vgg-16，从一张噪音图开始，一方面比对content image，另一方面比对style image，不断调整噪音图，从而得到一张融合后的图。而Percept Loss For Real Time Transfer则是增加了一个auto encoder形状的网络，叫做transformation network，进原图出来预测图，这个预测图与原图在内容做比对，与风格图在风格上做比对。在训练的时候，有若干张内容图，只有一张风格图。这样就可以训练transformation network一边顾及到恢复原图，一边顾及到贴近特定的风格。  
结果就是，一个transformation network就贴近一张风格图。这样一来，一张新的内容图进来以后，直接就可以推断出风格迁移后的图像。图像质量理论上跟原始版本style transfer不会有啥差异，只不过是可以提前训练好该模型，不像style transfer原始版本，每次都要训练。

首先，拿一系列内容图+一张风格图训练一个网络，并保存下来

In [1]:
from main import build_parser
from train import network_train
from test import network_test
import os
from pathlib import Path

# 当前目录
#current_directory = os.path.dirname(os.path.abspath(__file__))
current_directory = os.getcwd()


d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 训练模式
content_imagedir = os.path.join(current_directory, "../Images/original/")
style_imagedir = os.path.join(current_directory, "../Images/style-images/")
content_weight = 10
style_weight = 300
tv_weight = 3
max_iter = 10000
subdirname = "StyleImages" + str(content_weight) + "_" + str(style_weight) + "_" + str(tv_weight) + "_" + str(max_iter) + "/"

model_outputpath = os.path.join(current_directory, "trained_models/", subdirname)

style_files = os.listdir(style_imagedir)
for style_filename in style_files:
    parser = build_parser()
    style_imagepath = os.path.join(style_imagedir, style_filename)
    stylename_nosuffix = str(Path(style_filename).with_suffix("")) 
    # 当前模型输出目录
    curmodel_outputdir = os.path.join(model_outputpath, stylename_nosuffix, "./")
    print("OutputDir:" + curmodel_outputdir)
    # 创建该目录
    is_exist = os.path.exists(curmodel_outputdir)
    if not is_exist:
        os.makedirs(curmodel_outputdir)

    #style_imagepath = os.path.join(current_directory, "../Images/cartoon/0001.jpg")
    custom_args = [
        "--train-flag", "True",
        "--train-content", content_imagedir,
        "--train-style", style_imagepath,
        "--content-weight", str(content_weight),
        "--style-weight", str(style_weight),
        "--tv-weight", str(tv_weight),
        "--max-iter", str(max_iter),
        "--save-path", curmodel_outputdir
    ]
    args = parser.parse_args(custom_args)
    print(args)
    network_train(args)


OutputDir:e:\workspace\577-project\PerceptualLossForRealTimeTransfer\trained_models/StyleImages10_300_3_10000/ben_giles\./
Namespace(batchs=8, check_iter=100, content_layers=[15], content_weight=10.0, cropsize=None, cuda_device_no=0, imsize=256, lr=0.1, max_iter=10000, model_load_path=None, output='stylized.jpg', save_path='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\trained_models/StyleImages10_300_3_10000/ben_giles\\./', style_layers=[3, 8, 15, 22], style_weight=300.0, test_content=None, train_content='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/original/', train_flag=True, train_style='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/style-images/ben_giles.jpg', tv_weight=3.0, vgg_flag='vgg16')
device:cuda:0


d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sat Nov 25 15:35:32 2023: iteration: [0/10000/],	content_loss: 0.23,	style_loss: 0.38,	tv_loss: 0.01,	total_loss: 116.52,	
Sat Nov 25 15:36:43 2023: iteration: [100/10000/],	content_loss: 15.96,	style_loss: 8.92,	tv_loss: 0.11,	total_loss: 2836.62,	
Sat Nov 25 15:37:55 2023: iteration: [200/10000/],	content_loss: 22.64,	style_loss: 4.47,	tv_loss: 0.25,	total_loss: 1569.44,	
Sat Nov 25 15:39:07 2023: iteration: [300/10000/],	content_loss: 25.59,	style_loss: 2.01,	tv_loss: 0.37,	total_loss: 859.96,	
Sat Nov 25 15:40:19 2023: iteration: [400/10000/],	content_loss: 26.87,	style_loss: 1.05,	tv_loss: 0.47,	total_loss: 585.28,	
Sat Nov 25 15:41:31 2023: iteration: [500/10000/],	content_loss: 27.52,	style_loss: 0.68,	tv_loss: 0.52,	total_loss: 480.30,	
Sat Nov 25 15:42:42 2023: iteration: [600/10000/],	content_loss: 27.79,	style_loss: 0.54,	tv_loss: 0.54,	total_loss: 442.11,	
Sat Nov 25 15:44:13 2023: iteration: [700/10000/],	content_loss: 27.84,	style_loss: 0.47,	tv_loss: 0.54,	total_loss: 42

现在开始测试

In [ ]:
parser = build_parser()
model_path = os.path.join(current_directory, "trained_models/transform_network.pth")
test_imagepath = os.path.join(current_directory, "../Images/content-images/HeadBase.jpg")
output_imagepath = os.path.join(current_directory, "../Images/styletransfer-output/HeadBaseTransfered.jpg")
custom_args = [
    "--train-flag", "False",
    "--model-load-path", model_path,
    "--test-content", test_imagepath,
    "--output", output_imagepath
]

args = parser.parse_args(custom_args)
print(args)

network_test(args)
